## Notebook

* Objective: Read and explore the feedback SLA files produced by the NEMO OBS operator. In particular we check the julian day time variable.
* Latest update: 2023-11-10
* stephanie leroux, Datlas.

In [1]:
## standart libraries
import os,sys
import numpy as np

# xarray
import xarray as xr

# time
import datetime
import netCDF4
import cftime

# plot
import cartopy.crs as ccrs
import cartopy.feature as ccf
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# for jupyter notebook display

%matplotlib inline

# 0. Read files

In [15]:
# example year to read
YR=1993

# directory and file names ofthe files are on Jean Zay:

# Synthetic SLA Feedback files produced by NEMO
diri="/gpfsstore/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-IMHOTEP.GAIc-S/OBS/"
fili="eORCA025.L75-IMHOTEP.GAIc_y"+str(YR)+"m??d??_slafb_fdbk.???.nc"

# True SLA obs files that the NEMO-OBS operator takes as input (from AVISO)
dirin="/gpfsstore/rech/cli/commun/DATASET/OBS/AVISO/ALL/"
filin="fdbk_dt_global_SAT_phy_l3_"+str(YR)+"??.nc"


In [16]:
# load data with xarray
moddataset = xr.open_mfdataset(diri+fili,decode_times=False,combine='nested',concat_dim='N_OBS')
truedataset = xr.open_mfdataset(dirin+filin,decode_times=False,combine='nested',concat_dim='N_OBS')

# read NTOT the total number of record in the concatenated files
NTOT=int(moddataset['SLA_OBS'].N_OBS[-2:-1].values)

# print infos about the dataset
moddataset

<xarray.Dataset>
Dimensions:               (N_OBS: 16327223, N_VARS: 1, N_ENTRIES: 2,
                           N_EXTRA: 1, N_LEVELS: 1, N_QCF: 2)
Dimensions without coordinates: N_OBS, N_VARS, N_ENTRIES, N_EXTRA, N_LEVELS,
                                N_QCF
Data variables: (12/31)
    VARIABLES             (N_OBS, N_VARS) |S8 dask.array<chunksize=(1469596, 1), meta=np.ndarray>
    ENTRIES               (N_OBS, N_ENTRIES) |S8 dask.array<chunksize=(1469596, 2), meta=np.ndarray>
    EXTRA                 (N_OBS, N_EXTRA) |S8 dask.array<chunksize=(1469596, 1), meta=np.ndarray>
    STATION_IDENTIFIER    (N_OBS) |S8 dask.array<chunksize=(1469596,), meta=np.ndarray>
    STATION_TYPE          (N_OBS) |S4 dask.array<chunksize=(1469596,), meta=np.ndarray>
    LONGITUDE             (N_OBS) float64 dask.array<chunksize=(1469596,), meta=np.ndarray>
    ...                    ...
    SLA_LEVEL_QC_FLAGS    (N_OBS, N_LEVELS, N_QCF) int32 dask.array<chunksize=(1469596, 1, 2), meta=np.ndarray>
    SLA_IOBSI             (N_OBS) int32 dask.array<chunksize=(1469596,), meta=np.ndarray>
    SLA_IOBSJ             (N_OBS) int32 dask.array<chunksize=(1469596,), meta=np.ndarray>
    SLA_IOBSK             (N_OBS, N_LEVELS) int32 dask.array<chunksize=(1469596, 1), meta=np.ndarray>
    SLA_GRID              (N_OBS) |S1 b'T' b'T' b'T' b'T' ... b'T' b'T' b'T'
    MDT                   (N_OBS, N_LEVELS) float32 dask.array<chunksize=(1469596, 1), meta=np.ndarray>
Attributes:
    title:       NEMO observation operator output
    Convention:  NEMO unified observation operator output

* __Convert Julian to standard calendar (e.g., Gregorian)__

In [17]:
# Convert Julian to standard calendar (e.g., Gregorian)
caltime_true = xr.coding.times.decode_cf_datetime(truedataset.JULD, units="days since 1950-01-01T00:00",calendar='gregorian')
caltime_mod = xr.coding.times.decode_cf_datetime(moddataset.JULD, units="days since 1950-01-01T00:00",calendar='gregorian')

---
# 1. Compare julian day time
Compare julian day time in the feedback file produced by the model, and in the original AVISO file take as input by the NEMO-OBS operator.

In [18]:
moddataset.JULD[:50].values

array([15705.97222222, 15705.97222222, 15705.97222222, 15705.97222222,
       15705.97222222, 15705.97222222, 15705.97222222, 15705.97222222,
       15705.97222222, 15705.97222222, 15705.97222222, 15705.97222222,
       15705.97222222, 15705.97222222, 15705.97222222, 15705.97222222,
       15705.97222222, 15705.97222222, 15705.97222222, 15705.97222222,
       15705.97222222, 15705.97222222, 15705.97222222, 15705.97291667,
       15705.97291667, 15705.97291667, 15705.97291667, 15705.97291667,
       15705.97291667, 15705.97291667, 15705.97291667, 15705.97291667,
       15705.97291667, 15705.97291667, 15705.97291667, 15705.97291667,
       15705.97291667, 15705.97291667, 15705.97291667, 15705.97291667,
       15705.97291667, 15705.97291667, 15705.97291667, 15705.97291667,
       15705.97291667, 15705.97291667, 15705.97291667, 15705.97291667,
       15705.97291667, 15705.97291667])

In [39]:
truedataset.JULD[:50].values

array([15705.97263629, 15705.97264877, 15705.97266125, 15705.97267374,
       15705.97268622, 15705.97269871, 15705.97271119, 15705.97272367,
       15705.97273616, 15705.97274864, 15705.97276112, 15705.97277361,
       15705.97278609, 15705.97279857, 15705.97281106, 15705.97282354,
       15705.97283602, 15705.97284851, 15705.97286099, 15705.97287348,
       15705.97288596, 15705.97289844, 15705.97291093, 15705.97294838,
       15705.97296086, 15705.97299831, 15705.97301079, 15705.97302328,
       15705.97303576, 15705.97304825, 15705.97306073, 15705.97307321,
       15705.9730857 , 15705.97309818, 15705.97311066, 15705.97312315,
       15705.97313563, 15705.97314811, 15705.9731606 , 15705.97317308,
       15705.97318556, 15705.97319805, 15705.97321053, 15705.97322302,
       15705.9732355 , 15705.97324798, 15705.97326047, 15705.97327295,
       15705.97328543, 15705.97329792])

__--> The model julian day time from the file produced by the model is approximated (several entries have the exact same julian day values in moddataset).__

---
# 2. Compare calendar dates 
Compare calendar dates in the feedback file produced by the model, and in the original AVISO file take as input by the NEMO-OBS operator:

_How much is the julian day time approximated in the model files?_

In [19]:
caltime_mod[:50]

array(['1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:20:00.000000000',
       '1992-12-31T23:20:00.000000000', '1992-12-31T23:21:00.000000000',
       '1992-12-31T23:21:00.000000000', '1992-12-31T23:21:00.000000000',
       '1992-12-31T23:21:00.000000000', '1992-12-31

In [20]:
caltime_true[:50]

array(['1992-12-31T23:20:35.775270144', '1992-12-31T23:20:36.853850112',
       '1992-12-31T23:20:37.932430080', '1992-12-31T23:20:39.011010048',
       '1992-12-31T23:20:40.089590272', '1992-12-31T23:20:41.168170240',
       '1992-12-31T23:20:42.246750208', '1992-12-31T23:20:43.325330176',
       '1992-12-31T23:20:44.403910144', '1992-12-31T23:20:45.482490112',
       '1992-12-31T23:20:46.561070080', '1992-12-31T23:20:47.639650048',
       '1992-12-31T23:20:48.718230016', '1992-12-31T23:20:49.796810240',
       '1992-12-31T23:20:50.875390208', '1992-12-31T23:20:51.953970176',
       '1992-12-31T23:20:53.032550144', '1992-12-31T23:20:54.111130112',
       '1992-12-31T23:20:55.189710336', '1992-12-31T23:20:56.268290304',
       '1992-12-31T23:20:57.346870016', '1992-12-31T23:20:58.425450240',
       '1992-12-31T23:20:59.504030208', '1992-12-31T23:21:02.739770112',
       '1992-12-31T23:21:03.818350080', '1992-12-31T23:21:07.054090240',
       '1992-12-31T23:21:08.132670208', '1992-12-31

__->The julian day time in the model files is approximated below 1 min compared to the julian day time in the original AVISO files that also take into account seconds and decimals of seconds.__